<h1><font color='skyblue'>Spark Funds Investment Data Analysis EDA Assignment</font> </h1>

---

## Business Objective

The objective is to identify the best sectors, countries, and a suitable investment type for making investments. The overall strategy is to invest where others are investing, implying that the 'best' sectors and countries are the ones 'where most investors are investing’.

#### Business Condition
<li>
invest between 5 to 15 million USD per round of investment</li>
 <li>English-speaking countries
</li>


##### --------Let's Begin ----------#####

### 1. Data Understanding/Exploration

pd.read_csv function works normally but depends on the file we are parsing we need to use proper encoding and engine.
here you will get utf-8' codec can't decode byte 0xa0 in position 25: invalid start byte error which simply says encoding is wrong.
    https://docs.python.org/3/library/codecs.html#standard-encodings
    https://stackoverflow.com/questions/22216076/unicodedecodeerror-utf8-codec-cant-decode-byte-0xa5-in-position-0-invalid-s
    https://stackoverflow.com/questions/18171739/unicodedecodeerror-when-reading-csv-file-in-pandas-with-python
    https://docs.python.org/3.3/howto/unicode.html
    
    Then we will do some basic operation to get look and feel of the data
    - companies.head() -- Top 5 records 
    - companies.info() -- Basic details, column names, not nulls, type
    - rounds2.isnull().sum() -- To check total null values by column 
    - Describe() - to. check mean mode median etc data points

import chardet
with open("projects.csv", 'rb') as rawdata:
    result = chardet.detect(rawdata.read(10000))

--check what the character encoding might be good
print(result)
    

In [ ]:
import pandas as pd
import numpy as np
companies = pd.read_csv('../input/companydata/companies.csv',encoding= 'unicode_escape', engine = 'python')
rounds2 = pd.read_csv('../input/spark-fund-investment-analysis/datasets/rounds2.csv',encoding= 'unicode_escape', engine = 'python')

In [ ]:
companies.head()

In [ ]:
companies.info()

In [ ]:
rounds2.head()

In [ ]:
rounds2.info()

In [ ]:
rounds2.isnull().sum()

In [ ]:
companies.isnull().sum()

## Checkpoint 1: Data Cleaning 1


    1. How many unique companies are present in rounds2?
    2. How many unique companies are present in companies?                      
    3. In the companies data frame, which column can be used as the unique key for each company? 
       Write the name of the column.
    4. Are there any companies in the rounds2 file which are not present in companies? Answer yes or no: Y/N 
    5. Merge the two data frames so that all variables (columns) in the companies frame are added to the rounds2 
       data frame. Name the merged frame master_frame. How many observations are present in master_frame?
       
[NOTE:] As you can see permalink is the unique key for the table. to make you confuse there is company name as well. but if you see the example of Agora. you will find even though name is same but they deal with different category and got different funding type and amount.

[NOTE:] Lowercase everything to get unique 

[NOTE:] you will still see some encoding issue like aeron, asys round2 csv " ä°novatiff-reklam-ve-tanä±tä±m-hizmetleri-tic". below 2 links will give more knowledge on encoding decoding 
1. upGrad Prep Course Module 4 session 4
2. https://docs.python.org/3.3/howto/unicode.html

In [ ]:
rounds2['company_permalink'] = rounds2['company_permalink'].str.lower().str.encode('ISO-8859-1').str.decode('ascii','ignore')
companies['permalink'] = companies['permalink'].str.lower().str.encode('ISO-8859-1').str.decode('ascii','ignore')

In [ ]:
# How many unique companies are present in rounds2?
print(rounds2.company_permalink.unique().size)

# another way -1 len(set(rounds2['company_permalink'].values.tolist()))
# another way -2 rounds2['company_permalink'].describe()
# another way -3 duplicate_comp_rounds2 = rounds2[rounds2.duplicated('company_permalink')]
# len(rounds2)-len(duplicate_comp_rounds2)

In [ ]:
print(companies.permalink.unique().size)
len(set(companies['permalink'].values.tolist()))

#companies['company_permalink'].describe()
# duplicate_comp = companies[companies.duplicated('permalink')]
# duplicate_comp

In [ ]:
# Are there any companies in the rounds2 file which are not present in companies? Answer yes or no: Y/N -- N

differance = set(rounds2['company_permalink'].values.tolist())-set(companies['permalink'].values.tolist())
len(differance)
diff_companies_with_rounds = set(companies['permalink'].values.tolist()) - set(rounds2['company_permalink'].values.tolist())
len(diff_companies_with_rounds)

In [ ]:
#Merge the two data frames so that all variables (columns) in the companies frame are added to the rounds2 data frame. 
#Name the merged frame master_frame. How many observations are present in master_frame?
# Ans - 114949
master_frame = pd.merge(rounds2,companies, left_on=rounds2['company_permalink'], right_on=companies['permalink'], how='inner')
master_frame.shape

## Checkpoint 2: Funding Type Analysis

Spark Funds wants to choose one of these four investment types (seed, venture, angel, Private equity) for each potential investment they will make.

1. Calculate the most representative value of the investment amount for each of the four funding types (venture, angel, seed, and private equity)
2. Based on the most representative investment amount calculated above, which investment type do you think is the most suitable for Spark Funds?

Output: 
1. Representative funding amount of venture type
2. Representative funding amount of angel type
3. Representative funding amount of seed type
4. Representative funding amount of private equity type
5. Considering that Spark Funds wants to invest between 5 to 15 million USD per  investment round, which investment type is the most suitable for them?


Removing null/nan values
Filter master_frame for gievn investment type (seed, venture, angel, Private equity)

In [ ]:
master_frame.isnull().sum()

In [ ]:
# summing up the missing values (column-wise) and displaying fraction of NaNs
round(100*(master_frame.isnull().sum()/len(master_frame.index)), 2)

In [ ]:
# removing redundant columns
master_frame =  master_frame.drop(['key_0','state_code', 'region', 'city', 'founded_at','funding_round_permalink',
       'funding_round_code', 'funded_at','name','homepage_url','status'], axis=1)


In [ ]:
round(100*(master_frame.isnull().sum()/len(master_frame.index)), 2)

In [ ]:
master_frame = master_frame[master_frame.funding_round_type.isin(['venture','angel','seed','private_equity'])]
#master_frame_mrv_sub = master_frame_mrv.loc[:,['funding_round_type','raised_amount_usd']]
#master_frame_mrv_sub.head()
#master_frame_mrv_sub.groupby(master_frame_mrv_sub['funding_round_type'])['raised_amount_usd'].mean().round(1)
master_frame.head()

In [ ]:
round(100*(master_frame.isnull().sum()/len(master_frame.index)), 2)

In [ ]:
master_frame.raised_amount_usd.describe()

In [ ]:
# converting in mn$
master_frame['raised_amount_usd'] = master_frame['raised_amount_usd'].apply(lambda x:x/1000000).round(1)
master_frame['raised_amount_usd']

In [ ]:
master_frame.raised_amount_usd.describe()

In [ ]:
import matplotlib.pyplot as plt 
import seaborn as sns
%matplotlib inline

In [ ]:
master_frame.loc[~(master_frame.raised_amount_usd.isnull())]['raised_amount_usd']

In [ ]:
#Create a box plot for the raised_amount_usd column
plt.boxplot(master_frame.loc[~(master_frame.raised_amount_usd.isnull())]['raised_amount_usd'])
plt.show()

 Its look like we can not impute missing column with any value so better to remove them 
 
 

In [ ]:
master_frame = master_frame.loc[~(master_frame.raised_amount_usd.isnull() | master_frame.category_list.isnull() | master_frame.country_code.isnull() )]
master_frame.shape

In [ ]:
master_frame_funding_Type = master_frame
master_frame_funding_Type.shape

In [ ]:
round(100*(master_frame.isnull().sum()/len(master_frame.index)), 2)

In [ ]:
master_frame.funding_round_type.value_counts()

#Most representetive value -- it measn Mean()

1. Representative funding amount of venture type
2. Representative funding amount of angel type
3. Representative funding amount of seed type
4. Representative funding amount of private equity type
5. Considering that Spark Funds wants to invest between 5 to 15 million USD per  investment round, which investment type is the most suitable for them?


funding_round_type
angel              0.97
private_equity    73.94
seed               0.74
venture           11.72
Name: raised_amount_usd, dtype: float64

In [ ]:

master_frame.groupby(master_frame['funding_round_type'])['raised_amount_usd'].mean().round(2)

## Checkpoint 3: Country Analysis

---
1. Spark Funds wants to see the top nine countries which have received the highest total funding (across ALL sectors for the chosen investment type)

2. For the chosen investment type, make a data frame named top9 with the top nine countries (based on the total investment amount each country has received)

 

Identify the top three English-speaking countries in the data frame top9.

In [ ]:
#filter data for venture type only

master_frame_venture = master_frame.loc[(master_frame.funding_round_type == 'venture')]
master_frame_venture.shape

In [ ]:
top9 = pd.pivot_table(data=master_frame_venture,index="country_code", values="raised_amount_usd", aggfunc=sum)
top9 = top9.sort_values(by='raised_amount_usd', ascending=False)
top9 = top9.iloc[:9,:].round(1)
top9
#in mn

# master_frame_venture.loc[(master_frame_venture.country_code=='JPN')]['raised_amount_usd'].sum()/1000000

In [ ]:
top9.raised_amount_usd = top9.raised_amount_usd.astype("int64")
top9


In [ ]:
# English speaking countries. China we can removed from top 3

master_frame_venture_en_top3 = master_frame_venture.loc[(master_frame_venture.country_code=='USA')
                                                        |(master_frame_venture.country_code=='IND')
                                                        |(master_frame_venture.country_code=='GBR')]

In [ ]:
master_frame = master_frame_venture_en_top3
master_frame.shape

## Checkpoint 4: Sector Analysis 1

---

1. Extract the primary sector of each category list from the category_list column

2. Use the mapping file 'mapping.csv' to map each primary sector to one of the eight main sectors (Note that ‘Others’ is also considered one of the main sectors)

Expected Results: Code for a merged data frame with each primary sector mapped to its main sector (the primary sector should be present in a separate column).

[NOTE] : category_list has multiple values divided by |. so we need to manipulate it to the 1st value of the str array.

[NOTE] : Mapping data set is in  wide format so we need to Changing to long format with help of (pd.melt)

In [ ]:
master_frame['category_list'] = master_frame['category_list'].str.split('|').str[0].str.lower()
master_frame.shape
master_frame.info()

In [ ]:
import chardet
with open("../input/spark-fund-investment-analysis/datasets/mapping.csv", 'rb') as rawdata:
    result = chardet.detect(rawdata.read(10000))

# check what the character encoding might be
print(result)

In [ ]:
mapping = pd.read_csv('../input/spark-fund-investment-analysis/datasets/mapping.csv',encoding= 'ascii', engine = 'python')
#mapping['main_category'] = mapping.dot(mapping.columns+',').str[:-1]
mapping.head()

In [ ]:
mapping.info()

In [ ]:
mapping.category_list

#### Changing from wide format to long format pd.melt()

#### Parameters
    ----------
    id_vars : tuple, list, or ndarray, optional
        Column(s) to use as identifier variables.
    value_vars : tuple, list, or ndarray, optional
        Column(s) to unpivot. If not specified, uses all columns that
        are not set as `id_vars`.
        
[NOTE] : Same u can achieve by dot function as well

df1 = mapping.iloc[:,1:].dot(mapping.iloc[:,1:].columns)

mapping['main_category'] = df1

mapping = mapping.loc[:,['category_list','main_category']]



In [ ]:
wide_df = pd.DataFrame(mapping)
wide_df

# store the value and id variables in two separate arrays

# store the value variables in one Series
value_vars = list(wide_df.columns[1:])

# take the setdiff() to get the rest of the variables
id_vars = list(np.setdiff1d(wide_df.columns, value_vars))

print(value_vars, "\n")
print(id_vars)

In [ ]:
# convert into long
long_df = pd.melt(wide_df, 
        id_vars = id_vars, 
        value_vars = value_vars)
long_df

In [ ]:
# remove rows having value=0
long_df = long_df[long_df['value']==1]
long_df = long_df.drop('value', axis=1)
long_df

In [ ]:
# renaming the 'variable' column
long_df = long_df.rename(columns={'variable': 'main_sector'})
long_df

In [ ]:
mapping = long_df
mapping.category_list = mapping.category_list.str.lower()
mapping


In [ ]:
mapping.loc[(mapping.category_list == 'enterprise 2.na')]

#### As part of Data cleaning on Mapping Dataset

[NOTE]: when you analyze the mapping dataset  you will found some weird data it seems some issue in the data. like Analytics was A0lytics. 

when i did merge and check for null i found below result. how analytics come as NaN. then i checked my mapping file and found that Analytics was A0lytics

same thing was happening for some other category like Finance = Fi0nce means 0 = na 

Analytics                          1334
Finance                             561
Nanotechnology                      141
Financial Services                  105

#then 141 records for Nanotechnology got failed we have to lower the case for category_list

enterprise 2.0                     4
adaptive equipment                 2
retirement                         2
racing                             1

code to check after merging -- master_frame.loc[(master_frame.main_category.isnull())]['category_list'].value_counts()



In [ ]:
# Replacing 0 with na i.e A0lytics to Analytics
mapping.category_list = mapping.category_list.str.replace('0','na')
mapping.category_list = mapping.category_list.str.replace('2.na','2.0')

# Lower case for master_frame['category_list'] so that no merging issue 
master_frame['category_list']=master_frame['category_list'].str.lower()
master_frame = master_frame.merge(mapping, on='category_list', how='left')
master_frame.shape


In [ ]:
master_frame.info()

In [ ]:
master_frame.head()

In [ ]:
master_frame.loc[(master_frame.main_sector.isnull())]['category_list'].value_counts()


In [ ]:
master_frame.head()

In [ ]:
# the range of funding preferred by Spark Funds is 5 to 15 million USD
master_frame = master_frame.loc[(master_frame.raised_amount_usd<=15)&(master_frame.raised_amount_usd>=5)]
master_frame.shape

In [ ]:
master_frame.isnull().sum()

In [ ]:
## Removing null value

master_frame = master_frame.loc[~(master_frame.main_sector.isnull())]
master_frame.isnull().sum()

 ## Checkpoint 5: Sector Analysis 2
 
 ---
 
<li> Create three separate data frames D1, D2 and D3 for each of the three countries containing the observations of funding type FT falling within the 5-15 million USD range. 
    <li> The three data frames should contain all the columns of the master_frame along with the main sector and the primary sector. 
        <li> Using the three data frames, you can calculate the total number of investments and the total amount of investments in each main sector for each of the three countries.

In [ ]:
d1 = master_frame.loc[(master_frame.country_code=='USA')]
d2 = master_frame.loc[(master_frame.country_code=='GBR')]
d3 = master_frame.loc[(master_frame.country_code=='IND')]
d1.head()

In [ ]:
d2.head()

In [ ]:
d3.head()

1. Total number of investments (count)

2. Total amount of investment (USD)

3. Top sector (based on count of investments)

4. Second-best sector (based on count of investments)

5. Third-best sector (based on count of investments)

6. Number of investments in the top sector (refer to point 3)

7. Number of investments in the second-best sector (refer to point 4)

8. Number of investments in the third-best sector (refer to point 5)

9. For the top sector count-wise (point 3), which company received the highest investment?

10. For the second-best sector count-wise (point 4), which company received the highest investment?

In [ ]:
res_d1 = d1.groupby(d1['main_sector'])
res_d1
#d1 = d1.merge(res_d1,how='left', on='main_sector')
#d1

In [ ]:
#Total number of investments (count)

print(d1.main_sector.value_counts().sum())
print(d2.main_sector.value_counts().sum())
print(d3.main_sector.value_counts().sum())

In [ ]:
#d1['main_sector_raised_amount_usd'] = d1.groupby(d1['main_sector'])['raised_amount_usd'].sum()
res_d1 = d1.groupby(d1['main_sector'])['raised_amount_usd'].sum()

In [ ]:
res_d3 = d3.groupby(d3['main_sector'])['raised_amount_usd'].sum()

In [ ]:
res_d2 = d2.groupby(d2['main_sector'])['raised_amount_usd'].sum()

In [ ]:
#Total amount of investment (USD)

print(res_d1.sum().round(2))
print(res_d2.sum().round(2))
print(res_d3.sum().round(2))

In [ ]:
# Top sector (based on count of investments)

d1_pivot = pd.pivot_table(data=d1,values="raised_amount_usd",index="main_sector", aggfunc=[sum,'count'])
d1_pivot
#Others	26405.6	2961
#Social, Finance, Analytics, Advertising	23881.4	2723
# Cleantech / Semiconductors	21753.6	2364

In [ ]:
# Top second sector (based on count of investments)

d2_pivot = pd.pivot_table(data=d2,values="raised_amount_usd",index="main_sector", aggfunc=[sum,'count'])
d2_pivot
# Others	1303.9	149
# Social, Finance, Analytics, Advertising	1089.3	133
# Cleantech / Semiconductors	1169.1	131

In [ ]:
# Top third (based on count of investments)

d3_pivot = pd.pivot_table(data=d3,values="raised_amount_usd",index="main_sector", aggfunc=[sum,'count'])
d3_pivot
# Others	1013.4	110
# Social, Finance, Analytics, Advertising	550.5	60
# News, Search and Messaging	438.9	53

In [ ]:
# For the top sector count-wise (point 3), which company received the highest investment?

d1_comp_top1 = d1.loc[(d1.main_sector == 'Others')]

d1_comp_top1_pivot = pd.pivot_table(data=d1_comp_top1,values="raised_amount_usd",index="company_permalink", aggfunc=sum)

d1_comp_top1_pivot = d1_comp_top1_pivot.sort_values(by='raised_amount_usd', ascending= False)
print(d1_comp_top1_pivot.head(1))

# For the second top sector count-wise (point 3), which company received the highest investment?
d1_comp_top2 = d1.loc[(d1.main_sector == 'Social, Finance, Analytics, Advertising')]

d1_comp_top2_pivot = pd.pivot_table(data=d1_comp_top2,values="raised_amount_usd",index="company_permalink", aggfunc=sum)

d1_comp_top2_pivot = d1_comp_top2_pivot.sort_values(by='raised_amount_usd', ascending= False)
print(d1_comp_top2_pivot.head(1))


In [ ]:
d2_comp_top1 = d2.loc[(d2.main_sector == 'Others')]

d2_comp_top1_pivot = pd.pivot_table(data=d2_comp_top1,values="raised_amount_usd",index="company_permalink", aggfunc=sum)

d2_comp_top1_pivot = d2_comp_top1_pivot.sort_values(by='raised_amount_usd', ascending= False)
print(d2_comp_top1_pivot.head(1))

# For the second top sector count-wise (point 3), which company received the highest investment?
d2_comp_top2 = d2.loc[(d2.main_sector == 'Social, Finance, Analytics, Advertising')]

d2_comp_top2_pivot = pd.pivot_table(data=d2_comp_top2,values="raised_amount_usd",index="company_permalink", aggfunc=sum)

d2_comp_top2_pivot = d2_comp_top2_pivot.sort_values(by='raised_amount_usd', ascending= False)
print(d2_comp_top2_pivot.head(1))

In [ ]:
d3_comp_top1 = d3.loc[(d3.main_sector == 'Others')]

d3_comp_top1_pivot = pd.pivot_table(data=d3_comp_top1,values="raised_amount_usd",index="company_permalink", aggfunc=sum)

d3_comp_top1_pivot = d3_comp_top1_pivot.sort_values(by='raised_amount_usd', ascending= False)
print(d3_comp_top1_pivot.head(1))

# For the second top sector count-wise (point 3), which company received the highest investment?
d3_comp_top2 = d3.loc[(d3.main_sector == 'Social, Finance, Analytics, Advertising')]

d3_comp_top2_pivot = pd.pivot_table(data=d3_comp_top2,values="raised_amount_usd",index="company_permalink", aggfunc=sum)

d3_comp_top2_pivot = d3_comp_top2_pivot.sort_values(by='raised_amount_usd', ascending= False)
print(d3_comp_top2_pivot.head(1))

## Checkpoint 6: Plots

---

As a final step, you have to present your findings to the CEO of Spark Funds. Specifically, she wants to see the following plots:

 

1. A plot showing the representative amount of investment in each funding type. This chart should make it clear that a certain funding type (FT) is best suited for Spark Funds.   

2. A plot showing the top 9 countries against the total amount of investments of funding type FT. This should make the top 3 countries (Country 1, Country 2, and Country 3) very clear.                                                                                                                                                                             

3. A plot showing the number of investments in the top 3 sectors of the top 3 countries on one chart (for the chosen investment type FT). This plot should clearly display the top 3 sectors each in Country 1, Country 2, and Country 3.

In [ ]:
import matplotlib.pyplot as plt 
import seaborn as sns
%matplotlib inline

#### A plot showing the representative amount of investment in each funding type. This chart should make it clear that a certain funding type (FT) is best suited for Spark Funds.

In [ ]:

plot_bar_ft = master_frame_funding_Type.groupby(master_frame_funding_Type['funding_round_type'])['raised_amount_usd'].mean().round(2)
plot_bar_ft
##Plot a bar plot of Content Rating vs Average Rating 
#master_frame_funding_Type.groupby(['funding_round_type'])['raised_amount_usd'].mean().plot.barh()
#plt.show()

# adding Mn to the value
plot_bar_ft_val = plot_bar_ft.apply(lambda x : str(x) + ' Mn')
plot_bar_ft_val


In [ ]:
import plotly           #(version 4.5.4) #pip install plotly==4.5.4
import plotly.express as px
import plotly.io as pio



barchart = px.bar(
    data_frame=plot_bar_ft,
    x=plot_bar_ft.index,
    y=plot_bar_ft.values,
    color = plot_bar_ft.index,
    color_discrete_map={
                "angel": "skyblue",
                "seed": "skyblue",
                "venture": "orange",
                "private_equity": "skyblue"}, # differentiate color of marks
    #opacity=0.9,                  # set opacity of markers (from 0 to 1)
    orientation="v",              # 'v','h': orientation of the marks
    labels={"funding_round_type":"Funding Type",
    "y":"Avg Investment in Million USD"}, 
    title='Funding Type Analysis (Avg Investment Funding)',
    text=plot_bar_ft_val.values,
    template='presentation',
)
barchart.update_layout(uniformtext_minsize=5, uniformtext_mode='show'),
barchart.update_traces(textposition='auto'),
pio.show(barchart)

In [ ]:
top9_cntry_ft = master_frame_venture.groupby(
    master_frame_venture['country_code'])['raised_amount_usd'].sum().apply(lambda t: t/1000).round(2)
top9_cntry_ft = top9_cntry_ft.sort_values(ascending=False)
top9_cntry_ft = top9_cntry_ft[:9]
type(top9_cntry_ft)

top9_cntry_ft_val = top9_cntry_ft.apply(lambda x : str(x) + ' Bn')
top9_cntry_ft
top9_cntry_ft_val

#### A plot showing the top 9 countries against the total amount of investments of funding type FT. This should make the top 3 countries (Country 1, Country 2, and Country 3) very clear.

In [ ]:
# top9.plot.bar()
# plt.show()

skyblue = 'Country'
color = ['skyblue' for i in range(9)]

barchart = px.bar(
    data_frame=top9_cntry_ft,
    x=top9_cntry_ft.index,
    y=top9_cntry_ft.values,
    color = top9_cntry_ft.index,
    color_discrete_map={
                "USA": "orange",
                "GBR": "orange",
                "IND": "orange",
                "CHN": "red",
                "ISR": "skyblue",
                "DEU": "skyblue",
                "JPN": "skyblue",
                "FRA": "skyblue",
                "CAN": "skyblue"},
    # differentiate color of marks
    #opacity=0.9,                  # set opacity of markers (from 0 to 1)
    orientation="v",              # 'v','h': orientation of the marks
    labels={"country_code":"Countries",
    "y":"Total Investment in Billion USD"}, 
    title='Total Venture FT Investment(Billion USD) by Country',
    text=top9_cntry_ft_val.values,
    template='presentation',
)
barchart.update_layout(uniformtext_minsize=5, uniformtext_mode='show'),
barchart.update_traces(textposition='outside'),
pio.show(barchart)

####  A plot showing the number of investments in the top 3 sectors of the top 3 countries on one chart (for the chosen investment type FT). This plot should clearly display the top 3 sectors each in Country 1, Country 2, and Country 3.

In [ ]:
# d1_pivot['count'].plot.bar()
# plt.show()
# d2_pivot['count'].plot.bar()
# plt.show()
# d3_pivot['count'].plot.bar()
# plt.show()



In [ ]:
d1_USA_Venture_Main_Sector = d1.groupby(d1['main_sector'])['funding_round_type'].count()
d1_USA_Venture_Main_Sector = d1_USA_Venture_Main_Sector.sort_values(ascending=False)
d1_USA_Venture_Main_Sector

In [ ]:
# Best Sector - USA (By Count of Investment
barchart = px.bar(
    data_frame=d1_USA_Venture_Main_Sector,
    x=d1_USA_Venture_Main_Sector.index,
    y=d1_USA_Venture_Main_Sector.values,
    color=d1_USA_Venture_Main_Sector.index,
    color_discrete_map={
                "Social, Finance, Analytics, Advertising": "green",
                "News, Search and Messaging": "skyblue",
                "Others": "skyblue",
                "Entertainment": "skyblue",
                "Manufacturing": "skyblue",
                "Health": "skyblue",
                "Cleantech / Semiconductors": "green",
                "Automotive & Sports": "skyblue"},
    # differentiate color of marks
    #opacity=0.9,                  # set opacity of markers (from 0 to 1)
    orientation="v",              # 'v','h': orientation of the marks
    labels={"main_sector":"Main Sectors",
    "y":"Total Count of Investment"}, 
    title='Best Sector By Count of Investment - USA',
    text=d1_USA_Venture_Main_Sector.values,
    template='plotly',
)   
pio.show(barchart)


In [ ]:
d2_GBR_Venture_Main_Sector = d2.groupby(
    d2['main_sector'])['funding_round_type'].count()
d2_GBR_Venture_Main_Sector = d2_GBR_Venture_Main_Sector.sort_values(ascending=False)
d2_GBR_Venture_Main_Sector

In [ ]:
barchart = px.bar(
    data_frame=d2_GBR_Venture_Main_Sector,
    x=d2_GBR_Venture_Main_Sector.index,
    y=d2_GBR_Venture_Main_Sector.values,
    color=d2_GBR_Venture_Main_Sector.index,
    color_discrete_map={
                "Social, Finance, Analytics, Advertising": "green",
                "News, Search and Messaging": "skyblue",
                "Others": "skyblue",
                "Entertainment": "skyblue",
                "Manufacturing": "skyblue",
                "Health": "skyblue",
                "Cleantech / Semiconductors": "green",
                "Automotive & Sports": "skyblue"},# differentiate color of marks
    #opacity=0.9,                  # set opacity of markers (from 0 to 1)
    orientation="v",              # 'v','h': orientation of the marks
    labels={"main_sector":"Main Sectors",
    "y":"Total Count of Investment"}, 
    title='Best Sector By Count of Investment - GBR',
    text=d2_GBR_Venture_Main_Sector.values,
   # width=600,                   # figure width in pixels
    #height=420, 
    template='plotly',
)   
pio.show(barchart)

In [ ]:
d3_IND_Venture_Main_Sector = d3.groupby(
    d3['main_sector'])['funding_round_type'].count()
d3_IND_Venture_Main_Sector = d3_IND_Venture_Main_Sector.sort_values(ascending=False)
d3_IND_Venture_Main_Sector

In [ ]:
barchart = px.bar(
    data_frame=d3_IND_Venture_Main_Sector,
    x=d3_IND_Venture_Main_Sector.index,
    y=d3_IND_Venture_Main_Sector.values,
    color=d3_IND_Venture_Main_Sector.index,
    color_discrete_map={
                "Social, Finance, Analytics, Advertising": "green",
                "News, Search and Messaging": "green",
                "Others": "skyblue",
                "Entertainment": "skyblue",
                "Manufacturing": "skyblue",
                "Health": "skyblue",
                "Cleantech / Semiconductors": "skyblue",
                "Automotive & Sports": "skyblue"},# differentiate color of marks
    #opacity=0.9,                  # set opacity of markers (from 0 to 1)
    orientation="v",              # 'v','h': orientation of the marks
    labels={"main_sector":"Main Sectors",
    "y":"Total Count of Investment"}, 
    title='Best Sector By Count of Investment - IND',
    text=d3_IND_Venture_Main_Sector.values,
    template='plotly',
)   
pio.show(barchart)

## Conclusion

---

Lets extract some information from the dataset which will help in the PPT

In [ ]:
# Calculate Year range for which we are analyzing dataset. it will tell us the given data set has teh range of this year to analyze

date = companies['founded_at'].loc[~(companies['founded_at'].isnull())].values
year = list(map(lambda x: int(str(x).split('-')[-1]), date))

year = list(map(lambda x: x if 1800 < x < 2020 else 2000, year))
year
print(min(year))
print(max(year))



In [ ]:
# to check how many discrete Countries to Analyze

companies['country_code'].unique().size